In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
assessments_raw = pd.read_csv("dataset/assessments.csv")
courses_raw = pd.read_csv("dataset/courses.csv")
student_assessment_raw = pd.read_csv("dataset/studentAssessment.csv")
student_info_raw = pd.read_csv("dataset/studentInfo.csv")
student_registration_raw = pd.read_csv("dataset/studentRegistration.csv")
# If this is failing, ensure that you have downloaded the dataset and copied studentVle.csv 
# to the dataset folder.
student_vle_raw = pd.read_csv("dataset/studentVle.csv") 
vle_raw = pd.read_csv("dataset/vle.csv")

In [4]:
print(student_assessment_raw.columns)
print(assessments_raw.columns)


Index(['id_assessment', 'id_student', 'date_submitted', 'is_banked', 'score'], dtype='object')
Index(['code_module', 'code_presentation', 'id_assessment', 'assessment_type',
       'date', 'weight'],
      dtype='object')


In [5]:
# read in existing wideform data
df_wide = pd.read_excel('wideform.xlsx')
df_wide.head()

,code_module,code_presentation,id_student,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,final_result,...,subpage_16-30,subpage_31-45,subpage_46-60,url_pre-0,url_1-15,url_16-30,url_31-45,url_46-60,sum_click_pre_A1,sum_click_pre_A2
0,AAA,2013J,11391,0,3,2,0,240,0,0,...,2,1,0,0,1,0,0,0,352,175
1,AAA,2013J,28400,1,3,1,0,60,0,0,...,16,1,1,10,9,11,1,1,493,156
2,AAA,2013J,31604,1,2,1,0,60,0,0,...,17,17,3,3,10,8,12,3,390,371
3,AAA,2013J,32885,1,1,0,0,60,0,0,...,8,0,1,3,0,3,0,0,558,64
4,AAA,2013J,38053,0,2,1,0,60,0,0,...,11,3,3,1,4,6,2,3,614,395


In [6]:
#weighted_score_A1
#weighted_score_A2
#df_wide[['weighted_score_A1','weighted_score_A2']].head()

In [7]:
# create for each module-presentation the weight for the first two TMAs
tma_assessments = assessments_raw[assessments_raw['assessment_type']=='TMA']
module_presentations = tma_assessments[['code_module','code_presentation']].drop_duplicates()
modules = tma_assessments[['code_module']].drop_duplicates()

df_first_two_tmas = tma_assessments.sort_values(by='date').groupby(['code_module','code_presentation']).head(2).sort_values(by=['code_module','code_presentation'])
weighted_possible_scores = df_first_two_tmas[['code_module','code_presentation','weight']].groupby(['code_module','code_presentation']).sum().reset_index()
weighted_possible_scores

,code_module,code_presentation,weight
0,AAA,2013J,30.0
1,AAA,2014J,30.0
2,BBB,2013B,23.0
3,BBB,2013J,23.0
4,BBB,2014B,23.0
5,BBB,2014J,10.0
6,CCC,2014B,31.0
7,CCC,2014J,31.0
8,DDD,2013B,17.5
9,DDD,2013J,22.5


In [8]:
#df_first_two_tmas
# tmas per module
#tma_assessments.groupby(['code_module','code_presentation']).count()
tma_assessments[['code_module','code_presentation','date']].drop_duplicates()

,code_module,code_presentation,date
0,AAA,2013J,19.0
1,AAA,2013J,54.0
2,AAA,2013J,117.0
3,AAA,2013J,166.0
4,AAA,2013J,215.0
...,...,...,...
193,GGG,2014B,117.0
194,GGG,2014B,166.0
202,GGG,2014J,61.0
203,GGG,2014J,124.0


In [9]:
cma_assessments = assessments_raw[assessments_raw['assessment_type']=='CMA']
print(cma_assessments.shape)
cma_assessments[['code_module','code_presentation','date']].drop_duplicates().shape

(76, 6)


(37, 3)

In [10]:
# do basic train-test split (80/20)
from sklearn.model_selection import train_test_split
df = df_wide.copy()
weighted_possible_scores = weighted_possible_scores.rename(columns={'weight':'out_of'})
df = df.merge(weighted_possible_scores, on=['code_module','code_presentation'])

df['grade'] = (df['weighted_score_A1'] + df['weighted_score_A2'])/(df['out_of'])

rule_based_columns = ['code_module','code_presentation','id_student','grade']
label = ['final_result']

X_train, X_test, y_train, y_test = train_test_split(df[rule_based_columns],df[label], test_size=0.2, random_state=42)

In [11]:
# test rule based method
def pred_rule_based(df):
    df['pred'] = df.apply(lambda x: 0 if x['grade']>0.4 else 1, axis=1)
    return df

train_preds = pred_rule_based(X_train)
train_preds['actual'] = y_train

In [12]:
# test scoring:
from sklearn.metrics import accuracy_score

In [13]:
score = accuracy_score(y_train,train_preds['pred'])
score

0.6436514522821577

In [14]:
# find accuracy by module
test_preds = pred_rule_based(X_test)
test_preds['actual'] = y_test

# print(test_preds.head())

accuracies = {}
for _,mp in module_presentations.iterrows():
    # select only for this course
    df_ = test_preds[(test_preds['code_module']==mp['code_module']) & (test_preds['code_presentation']==mp['code_presentation'])]
    accuracies[(mp['code_module'],mp['code_presentation'])] = np.round(accuracy_score(df_['actual'], df_['pred']),3)

accuracies

c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\num

{('AAA', '2013J'): 0.843,
 ('AAA', '2014J'): 0.743,
 ('BBB', '2013B'): 0.645,
 ('BBB', '2013J'): 0.696,
 ('BBB', '2014B'): 0.68,
 ('BBB', '2014J'): 0.668,
 ('CCC', '2014B'): 0.652,
 ('CCC', '2014J'): 0.592,
 ('DDD', '2013B'): 0.598,
 ('DDD', '2013J'): 0.608,
 ('DDD', '2014B'): 0.584,
 ('DDD', '2014J'): 0.605,
 ('EEE', '2013J'): 0.851,
 ('EEE', '2014B'): 0.708,
 ('EEE', '2014J'): 0.747,
 ('FFF', '2013B'): 0.616,
 ('FFF', '2013J'): 0.664,
 ('FFF', '2014B'): 0.549,
 ('FFF', '2014J'): 0.636,
 ('GGG', '2013J'): nan,
 ('GGG', '2014B'): nan,
 ('GGG', '2014J'): nan}

In [15]:
test_preds[(test_preds['code_module']=='EEE') & (test_preds['code_presentation']=='2013J')].shape

(175, 6)

In [16]:
all_preds = pd.concat([train_preds, test_preds])
accuracies_all = {}
for _,mp in module_presentations.iterrows():
    # select only for this course
    df_ = all_preds[(all_preds['code_module']==mp['code_module']) & (all_preds['code_presentation']==mp['code_presentation'])]
    accuracies_all[(mp['code_module'],mp['code_presentation'])] = np.round(accuracy_score(df_['actual'], df_['pred']),3)

accuracies_all

c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\num

{('AAA', '2013J'): 0.791,
 ('AAA', '2014J'): 0.774,
 ('BBB', '2013B'): 0.648,
 ('BBB', '2013J'): 0.687,
 ('BBB', '2014B'): 0.661,
 ('BBB', '2014J'): 0.687,
 ('CCC', '2014B'): 0.605,
 ('CCC', '2014J'): 0.612,
 ('DDD', '2013B'): 0.572,
 ('DDD', '2013J'): 0.603,
 ('DDD', '2014B'): 0.585,
 ('DDD', '2014J'): 0.576,
 ('EEE', '2013J'): 0.791,
 ('EEE', '2014B'): 0.738,
 ('EEE', '2014J'): 0.758,
 ('FFF', '2013B'): 0.608,
 ('FFF', '2013J'): 0.631,
 ('FFF', '2014B'): 0.582,
 ('FFF', '2014J'): 0.643,
 ('GGG', '2013J'): nan,
 ('GGG', '2014B'): nan,
 ('GGG', '2014J'): nan}

In [17]:
# accuracy on all
np.round(accuracy_score(all_preds['actual'],all_preds['pred']),3)

0.646

In [18]:
accuracies = {}
for _,mp in modules.iterrows():
    df_ = test_preds[(test_preds['code_module']==mp['code_module'])]
    accuracies[mp['code_module']] = np.round(accuracy_score(df_['actual'], df_['pred']),3)

accuracies

c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'AAA': 0.793,
 'BBB': 0.675,
 'CCC': 0.615,
 'DDD': 0.6,
 'EEE': 0.776,
 'FFF': 0.623,
 'GGG': nan}

In [19]:
accuracies_all = {}
for _,mp in modules.iterrows():
    df_ = all_preds[(all_preds['code_module']==mp['code_module'])]
    accuracies_all[mp['code_module']] = np.round(accuracy_score(df_['actual'], df_['pred']),3)

accuracies_all

c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
c:\Users\Mike\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'AAA': 0.783,
 'BBB': 0.673,
 'CCC': 0.61,
 'DDD': 0.586,
 'EEE': 0.765,
 'FFF': 0.62,
 'GGG': nan}

In [22]:
accuracies_single = np.round(accuracy_score(all_preds['actual'],all_preds['pred']),3)
accuracies_single

0.646

In [29]:
#student_info_raw['imd_band'].value_counts()
print(student_info_raw.shape[0])
print(len(student_info_raw['imd_band']))

32593
32593


In [43]:
#student_assessment_raw[student_assessment_raw['score'].isna()]
exams = assessments_raw[assessments_raw.isna().any(axis=1)][['id_assessment','code_module','code_presentation']]
merged_ = exams.merge(student_assessment_raw, on='id_assessment')
merged_['code_module'].value_counts()


CCC    1915
DDD     950
Name: code_module, dtype: int64

In [58]:
pass_pct = {}
modules = student_info_raw['code_module'].unique()
for mod in modules:
    #print(mod)
    val_counts = student_info_raw[student_info_raw['code_module']==mod]['final_result'].value_counts()
    pass_pct[mod] = (val_counts['Pass'] + val_counts['Distinction'])/(val_counts.sum())
pass_pct

{'AAA': 0.7098930481283422,
 'BBB': 0.47464913389809077,
 'CCC': 0.37843933243121336,
 'DDD': 0.4161352040816326,
 'EEE': 0.5623721881390593,
 'FFF': 0.4699819634114919,
 'GGG': 0.5974743488555643}

In [49]:
(487+44)/(487+126+91+44)

0.7098930481283422

In [66]:
# compare number of students per course affected by improved model
vcs = student_info_raw['code_module'].value_counts()
vcs = vcs.drop('GGG')
rf_acc = {"AAA":.787, "BBB":.746,"CCC":.815,"DDD":.786,"EEE":.863,"FFF":.75}
rb_acc = {"AAA":.783, "BBB":.673,"CCC":.610,"DDD":.586,"EEE":.765,"FFF":.646}
num_extra = {}
for k,v in vcs.items():
    num_extra[k] = v*(rf_acc[k]-rb_acc[k])
print(num_extra)
np.sum(list(num_extra.values()))


{'BBB': 577.3569999999996, 'FFF': 807.2479999999998, 'DDD': 1254.4000000000003, 'CCC': 908.9699999999998, 'EEE': 287.5319999999999, 'AAA': 2.9920000000000027}


3838.499